<a href="https://colab.research.google.com/github/JungMinNoh/jungminnoh/blob/master/2019_10_15_%EB%B0%91%EB%B0%94%EB%8B%A5_%EB%94%A5%EB%9F%AC%EB%8B%9D_5_4~5%EC%9E%A5_%EB%81%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 5.4 단순한 계층 구현하기 -14108334 노정민

In [0]:
class MulLayer:
    def __init__(self):
        self.x = None  # __init__ ()에서는 인스턴스 변수인 x 와 y를 초기화 한다.
        self.y = None

    def forward(self, x, y):
        self.x = x    # forward() 에서는 x와 y를 인수로 받고 두 값을 곱해서 반환한다.
        self.y = y                
        out = x * y

        return out

    def backward(self, dout):
        dx = dout * self.y  # x와 y를 바꾼다. # dout(미분) backward()는 상류에서 넘어온 순전파떄의 값을 서로 바꿔 곱한후 하류로 흘린다.
        dy = dout * self.x

        return dx, dy

In [0]:
apple = 100 
apple_num = 2
tax = 1.1

#계층들
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

# forward #순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

# backward #역전파 #역전파가 받는 인수는 순전파 출력에 대한 미분이다.
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice) 
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print("price:", int(price)) #price 값을 정수로
print("dApple:", dapple) 
print("dApple_num:", int(dapple_num))
print("dTax:", dtax)

price: 220
dApple: 2.2
dApple_num: 110
dTax: 200


# 5.4.2 덧셈 계층

In [0]:
class AddLayer:
    def __init__(self):
        pass #덧셈 계층에서는 초기화가 필요 없으니 __init__에서는 아무일도 하지 않는다(pass)

    def forward(self, x, y):
        out = x + y

        return out

    def backward(self, dout):
        dx = dout * 1  # 덧셈에서는 상류에서 내려온 미분을 그대로 하류로 흘릴뿐이다..
        dy = dout * 1

        return dx, dy

In [0]:
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

# layer #계층들 4개를 정해준다.
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

# forward 
apple_price = mul_apple_layer.forward(apple, apple_num)  # (1)
orange_price = mul_orange_layer.forward(orange, orange_num)  # (2)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)  # (3)
price = mul_tax_layer.forward(all_price, tax)  # (4)

# backward
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice)  # (4)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)  # (3)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)  # (2)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)  # (1)

print("price:", int(price))
print("dApple:", dapple)
print("dApple_num:", int(dapple_num))
print("dOrange:", dorange)
print("dOrange_num:", int(dorange_num))
print("dTax:", dtax)

price: 715
dApple: 2.2
dApple_num: 110
dOrange: 3.3000000000000003
dOrange_num: 165
dTax: 650


# 5.5 활성화 함수 계층 구현하기

In [0]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [0]:
import sys, os
sys.path.insert(0, '/content/gdrive/My Drive/Colab Notebooks/') # 부모 디렉터리의 파일을 가져올 수 있도록 설정
import numpy as np
from common.functions import *
from common.util import im2col, col2im


class Relu:
    def __init__(self):
        self.mask = None

    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0

        return out

    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout

        return dx


In [0]:
import numpy as np
x = np.array ([[1.0,-0.5],[-2.0,3.0]])
print(x)


[[ 1.  -0.5]
 [-2.   3. ]]


In [0]:
mask = (x<=0)
print(mask)

[[False  True]
 [ True False]]


# 5.5.2 Sigmoid 계층

In [0]:
class Sigmoid:
    def __init__(self):
        self.out = None

    def forward(self, x):
        out = sigmoid(x)  #순전파의 출력을 인스턴스 변수인 out에 보관했다가 역전파 계산떄 그 값을 사용.
        self.out = out 
        return out 

    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out

        return dx

# 5.6 Affine/Softmax 계층 구현하기

## 5.6.1 Affine 계층

In [0]:
X = np.random.rand(2) #입력
W = np.random.rand(2,3) #가중치
B = np.random.rand(3) #편향

X.shape #(2,)
W.shape #(2,3)
B.shape #(3,)

Y = np.dot(X,W) + B

## 5.6.2 배치용 Affine 계층

순전파의 편향 덧셈

In [0]:
X_dot_W = np.array([[0,0,0],[10,10,10]])
B = np.array([1,2,3])
X_dot_W


array([[ 0,  0,  0],
       [10, 10, 10]])

In [0]:
X_dot_W + B

array([[ 1,  2,  3],
       [11, 12, 13]])

역전파의 편향 덧셈

In [0]:
dY = np.array([[1,2,3], [4,5,6]])
dY

array([[1, 2, 3],
       [4, 5, 6]])

In [0]:
dB = np.sum(dY, axis=0)
dB

array([5, 7, 9])

In [0]:
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        
        self.x = None
        self.original_x_shape = None
        # 가중치와 편향 매개변수의 미분
        self.dW = None
        self.db = None

    def forward(self, x):
        # 텐서 대응
        self.original_x_shape = x.shape
        x = x.reshape(x.shape[0], -1)
        self.x = x

        out = np.dot(self.x, self.W) + self.b

        return out

    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        
        dx = dx.reshape(*self.original_x_shape)  # 입력 데이터 모양 변경(텐서 대응)
        return dx

## 5.6.3 Softmax-with-Loss 계층

In [0]:
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None # 손실함수
        self.y = None    # softmax의 출력
        self.t = None    # 정답 레이블(원-핫 인코딩 형태)
        
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        
        return self.loss

    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        if self.t.size == self.y.size: # 정답 레이블이 원-핫 인코딩 형태일 때
            dx = (self.y - self.t) / batch_size
        else:
            dx = self.y.copy()
            dx[np.arange(batch_size), self.t] -= 1
            dx = dx / batch_size
        
        return dx

# 5.7 오차역전파법 구현하기

전제  
  
 신경망에는 적응 가능한 가중치와 편향이 있고, 이 가중치와 편향을 훈련 데이터에 적응하도록 조정하는 과정을 '학습'이라 한다. 신경망 학습은 다음과 같이 4단계로 수행한다.  
 
1단계 - 미니배치  
 훈련 데이터 중 일부를 무작위로 가져온다. 이렇게 선별한 데이터를 미니 배치라 하며, 그 미니 배치의 손실 함수값을 줄이는것이 목표이다.  
 
2단계 - 기울기 산출  
 미니 배치의 손실 함수 값을 줄이기 위해 각 가중치 매개변수의 기울기를 구한다. 기울기는 손심 함수의 값을 가장 작게 하는 방향을 제시한다.  
  
3단계 - 매개 변수 갱신

가중치 매개 변수를 기울기 방향으로 아주 조금 갱신 한다.  

4단계 - 반복

1~3단계를 반복한다.



지금까지 설명한 오차역전파법이 등장하는 단계는 두번째인 '기울기 산출' 이다. 앞장에서는 이 기울기를 구하기 위해 수치 미분을 사용했다. 그런데 수치 미분은 구현하게 쉽지만, 계산이 오래 걸린다는 단점이 있었다. 오차역전파법을 이용하면 느린 수치 미분과 달리 기울기를 효율적이고 빠르게 구할 수 있다.


## 5.7.2 오차역전파법을 적용한 신경망 구현하기

In [0]:
# coding: utf-8
import sys, os
sys.path.insert(0, '/content/gdrive/My Drive/Colab Notebooks/')  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
import numpy as np
from common.layers import *
from common.gradient import numerical_gradient
from collections import OrderedDict


class TwoLayerNet:

    def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01):
        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size) 
        self.params['b2'] = np.zeros(output_size)

        # 계층 생성
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])

        self.lastLayer = SoftmaxWithLoss()
        
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        
        return x
        
    # x : 입력 데이터, t : 정답 레이블
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1 : t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
        
    # x : 입력 데이터, t : 정답 레이블
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads
        
    def gradient(self, x, t):
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.lastLayer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 결과 저장
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W2'], grads['b2'] = self.layers['Affine2'].dW, self.layers['Affine2'].db

        return grads

# 5.7.3 오차역전파법으로 구한 기울기 검증하기

In [0]:
# coding: utf-8
import sys, os
sys.path.insert(0, '/content/gdrive/My Drive/')  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
import numpy as np
from dataset.mnist import load_mnist
from two_layer_net import TwoLayerNet

# 데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

x_batch = x_train[:3]
t_batch = t_train[:3]

grad_numerical = network.numerical_gradient(x_batch, t_batch)
grad_backprop = network.gradient(x_batch, t_batch)

# 각 가중치의 절대 오차의 평균을 구한다.
for key in grad_numerical.keys():
    diff = np.average( np.abs(grad_backprop[key] - grad_numerical[key]) )
    print(key + ":" + str(diff))

W1:1.6358265856984743e-10
b1:8.260620979284384e-10
W2:7.055411274212165e-08
b2:1.4137802999691917e-07


# 5.7.4 오차역전파법을 사용한 학습 구현하기

In [0]:
# coding: utf-8
import sys, os
sys.path.insert(0, '/content/gdrive/My Drive/')

import numpy as np
from dataset.mnist import load_mnist
from two_layer_net import TwoLayerNet

# 데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    # 기울기 계산
    #grad = network.numerical_gradient(x_batch, t_batch) # 수치 미분 방식
    grad = network.gradient(x_batch, t_batch) # 오차역전파법 방식(훨씬 빠르다)
    
    # 갱신
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
    
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(train_acc, test_acc)

0.11236666666666667 0.1135
0.7787 0.7866
0.8744 0.8765
0.8983666666666666 0.9015
0.90825 0.9106
0.9138666666666667 0.9162
0.91965 0.9192
0.9241166666666667 0.9238
0.9282166666666667 0.9289
0.9320833333333334 0.932
0.9336333333333333 0.9344
0.9376166666666667 0.9378
0.9399 0.938
0.9415166666666667 0.9409
0.9444333333333333 0.9437
0.9450833333333334 0.9443
0.9486 0.9463
